<a href="https://colab.research.google.com/github/innsvi/python_for_ds_task/blob/main/hw_10_4_apply%2C_groupby%2C_pivot_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Домашнє завдання до тем apply(), groupby(), pivot_table()

В цьому домашньому завданні продовжуємо працювати з набором даних `supermarket_sales.csv`.

0. Імпортуйте бібліотеку pandas та зчитайте дані у змінну `df` типу `pandas.DataFrame`.

In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/data/supermarket_sales.csv')
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [6]:
print(df.head(5))

    Invoice ID Branch       City Customer type  Gender  \
0  750-67-8428      A     Yangon        Member  Female   
1  226-31-3081      C  Naypyitaw        Normal  Female   
2  631-41-3108      A     Yangon        Normal    Male   
3  123-19-1176      A     Yangon        Member    Male   
4  373-73-7910      A     Yangon        Normal    Male   

             Product line  Unit price  Quantity   Tax 5%     Total       Date  \
0       Health and beauty       74.69         7  26.1415  548.9715   1/5/2019   
1  Electronic accessories       15.28         5   3.8200   80.2200   3/8/2019   
2      Home and lifestyle       46.33         7  16.2155  340.5255   3/3/2019   
3       Health and beauty       58.22         8  23.2880  489.0480  1/27/2019   
4       Sports and travel       86.31         7  30.2085  634.3785   2/8/2019   

    Time      Payment    cogs  gross margin percentage  gross income  Rating  
0  13:08      Ewallet  522.83                 4.761905       26.1415     9.1  
1  10:

1. Дослідимо, який філіал супермаркету ('Branch') є найприбутковішим. Для цього знайдіть сумарний прибуток за кожним філіалом і виявіть, який філіал має найвищий.

In [7]:
bette_branch = df.groupby('Branch').sum()['gross income']
bette_branch

,gross income
Branch,
A,5057.1605
B,5057.0320
C,5265.1765


2. В якому місті знайходиться філіал з найвищим прибутком? Може в тому місці нам розмітисти ще один магазин.  
Знайдіть відповідь за допомогою функціоналу Pandas.

In [8]:
branch_profit = df.groupby(['Branch', 'City'])['gross income'].sum().reset_index()

branch_profit = branch_profit.sort_values(by='gross income', ascending=False)

most_profitable = branch_profit.iloc[0]

print(f"Найприбутковіший філіал: {most_profitable['Branch']}")
print(f"Місто: {most_profitable['City']}")
print(f"Загальний прибуток: {most_profitable['gross income']:.2f}")

Найприбутковіший філіал: C
Місто: Naypyitaw
Загальний прибуток: 5265.18


In [9]:

print("\nПорівняння всіх філіалів:")
print(branch_profit)


Порівняння всіх філіалів:
  Branch       City  gross income
2      C  Naypyitaw     5265.1765
0      A     Yangon     5057.1605
1      B   Mandalay     5057.0320


In [10]:
# порахуємо кількість чеків та середній чек для кожного міста
branch_stats = df.groupby('City').agg({
    'Invoice ID': 'count',      # кількість покупців
    'gross income': 'mean',     # середній прибуток з одного чека
    'Rating': 'mean'            # середній рейтинг
}).rename(columns={'Invoice ID': 'Transactions_Count'})

print(branch_stats)

           Transactions_Count  gross income    Rating
City                                                 
Mandalay                  332     15.232024  6.818072
Naypyitaw                 328     16.052367  7.072866
Yangon                    340     14.874001  7.027059


Якщо чесно, то з останніх даних виглядає так, шо відкривати мазазин у  Naypyitaw поки немає особливо сенсу, бо кількість транзакцій не набагато вища за інщі міста.

3.1. Створіть зводну таблицю, яка покаже, скільки покупок (інвойсів) було зроблено в кожній з філій (`Branch`) за різними категоріями товарів. Запишіть таблицю в змінну `invoices_by_category` і виведіть змінну на екран.
Ця таблиця допоможе проаналізувати, в якій філії купують найбільше товарів кожної з категорій.

In [11]:
invoices_by_category = df.pivot_table(index='Product line',
                                     columns='Branch',
                                     values='Invoice ID',
                                     aggfunc='count')

invoices_by_category

Branch,A,B,C
Product line,,,
Electronic accessories,60,55,55
Fashion accessories,51,62,65
Food and beverages,58,50,66
Health and beauty,47,53,52
Home and lifestyle,65,50,45
Sports and travel,59,62,45


Очікуваний результат:

![](https://drive.google.com/uc?export=view&id=1rueAdko6S3UxIHGtojetTxlES-EyM6Yb)

3.2. Викристовуючи змінну `invoices_by_category` дайте відповідь програмно (тобто значення треба не просто знайти очима, а вивести за допомогою коду), в якому філіалі магазину (`Branch`) найбільше інвойсів із покупкою товарів категорії "Електронні аксесуари" (`Electronic accessories`)?


In [12]:
best_branch_electronics = invoices_by_category.loc['Electronic accessories'].idxmax()

max_value = invoices_by_category.loc['Electronic accessories'].max()

print(f"Філіал з найбільшою кількістю продажів електроніки: {best_branch_electronics}")
print(f"Кількість інвойсів: {max_value}")

Філіал з найбільшою кількістю продажів електроніки: A
Кількість інвойсів: 60


4-6. **Творче завдання на розвиток аналітичного мислення**

Крок 1. Сформулюйте ТРИ питання (гіпотези) до наявних даних, які допомогли б вам зрозуміти, які користувачі що, де та коли найбільше/найменше купують, аби дати на основі цих гіпотез рекомендації бізнесу. Звісно питання мають бути не тими, на які ми вже відповіли в завданнях модулю.

Крок 2. Знайдіть відповіді на свої питання з допомогою функціоналу pandas.

Крок 3. Напишіть, як відповідь на це питання може бути використана для прийняття бізнес рішень.   
   
 Питання можуть бути будь-якої складності, але їх має бути 3. Кожне питання оцінюється як 1 завдання. Без виконання цього завдання ДЗ не приймається. Якщо є питання щодо виконання - пишіть у чат 🙌


ПИТАННЯ-ГІПОТЕЗА номер 1: В які дні тижня коден філіао має найбільшу кількість покупців. Маючи ці дані зможемо зробити рекомендації менеджменту, що потрібно в дні найбільшої заваниаженності вивести на роьоту більше касирів для збільшення пропускної здатності магазину і чеку відповідно.

In [13]:
df['Date'] = pd.to_datetime(df['Date'])
df['Day_of_Week'] = df['Date'].dt.day_name()

weekly_load = df.groupby(['Branch', 'Day_of_Week'])['Invoice ID'].count().unstack()

print("Завантаженість філіалів по днях тижня:")
weekly_load

Завантаженість філіалів по днях тижня:


Day_of_Week,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
Branch,,,,,,,
A,50,48,50,52,46,51,43
B,51,39,60,35,44,53,50
C,38,38,54,46,48,54,50


Філіал B має абсолютний пік у суботу (60 чеків). Це на 71% більше за його ж показник у неділю (35). Для бізнесу це сигнал, що у суботу в філіалі B мають працювати всі каси, тоді як у неділю частину персоналу можна відправити на вихідний або перенаправити на викладку товару.

ПИТАННЯ - ГІПОТЕЗА номер 2: Які дні є найбільш провальними для філіалів, і чи потребують вони стимулювання попиту?

In [14]:
low_days = weekly_load.idxmin(axis=1)
low_values = weekly_load.min(axis=1)

for branch, day in low_days.items():
    print(f"Філіал {branch}: Спад у {day} ({low_values[branch]} покупок)")

Філіал A: Спад у Wednesday (43 покупок)
Філіал B: Спад у Sunday (35 покупок)
Філіал C: Спад у Friday (38 покупок)


**Висновки:**

- Філіал B (Неділя — 35 покупок): Це найбільш критичний спад, оскільки в суботу цей же філіал має 60 покупок (майже вдвічі більше).

Рішення: Необхідно запровадити акцію «Сімейна неділя» або надати купони на знижку, які діють лише в неділю, щоб вирівняти потік клієнтів.

- Філіал C (П'ятниця — 38 покупок): Спад перед вихідними є нетиповим.

Рішення: Можливо, варто змінити асортимент товарів для відпочинку або провести промо-акцію «Готуйся до вихідних», щоб залучити людей у п'ятницю.

- Філіал A (Середа — 43 покупки): Найменший спад серед усіх філіалів.

Рішення: Цей філіал демонструє найвищу стабільність, тому тут можна проводити технічні роботи або інвентаризацію саме в середу з мінімальними втратами прибутку.

ПИТАННЯ - ГІПОТЕЗА номер 3: Чи приносять лояльні клієнти (Members) суттєво вищий середній чек, ніж звичайні покупці (Normal), у кожному з міст?

In [15]:
loyalty_impact = df.pivot_table(index='City',
                                columns='Customer type',
                                values='Total',
                                aggfunc='mean')

loyalty_impact['Benefit_%'] = ((loyalty_impact['Member'] - loyalty_impact['Normal']) / loyalty_impact['Normal'] * 100).round(2)

print("Аналіз середнього чека (Member vs Normal):")
loyalty_impact

Аналіз середнього чека (Member vs Normal):


Customer type,Member,Normal,Benefit_%
City,,,
Mandalay,325.482945,314.329257,3.55
Naypyitaw,336.575636,337.656755,-0.32
Yangon,321.182488,303.831763,5.71


- Yangon — успіх лояльності:
Тут програма працює найкраще: Benefit +5.71%. Постійні клієнти купують у середньому на 321.18, тоді як звичайні — на 303.83.

Рішення: Досвід Янгона варто масштабувати на інші міста.

- Naypyitaw — аномалія:тут ми бачимо від'ємний результат (-0.32%). Власники карток витрачають навіть трохи менше (336.57), ніж випадкові покупці (337.65).

Висновок: У цьому місті програма лояльності зараз не має фінансового сенсу. Можливо, бонуси не стимулюють до великих покупок, або клієнти використовують картку лише для отримання знижки на дрібниці.

- Mandalay: помірний приріст у 3.55%. Програма працює, але є куди рости.